- Tuning hyperparameters
  - Parsers = pypdf, pdfplumer(removed header footer)
  - Embedding model = FastEmbedEmbeddings(BAAI/bge-small-en-v1.5), BAAI/bge-large-en
  - Vectorstore = FAISS(fast), chromaDB(slow)
  - LLMs = llama3-70b-8192(robust), mixtral-8x7b-32768 (speciafic task)
  - Other hyperparameters = k, chunk_size
  - RAG Implemetations = (1) using FAISS retriver, (2) using parent document retriever

- Improvements
  - better embedding model
  - make it conversational
  - reranking, query transformation techniques

# Installing Dependencies and libraries

In [1]:
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
%%time
!pip install -q langchain
!pip install -q langchain-core
!pip install -q langchain-community
!pip install -q fastembed
!pip install -q pypdf
!pip install -q langchain_groq
!pip install -q faiss-gpu
!pip install -q sentence_transformers
!pip install -q chromadb
!pip install -q pdfplumber

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cud

In [6]:
import time
import pandas as pd
import numpy as np 
import random
import pdfplumber
from sklearn.metrics.pairwise import cosine_similarity
from langchain.chains.base import Chain
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS #Chroma
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.retrievers import ParentDocumentRetriever
from IPython.display import Markdown, display
# from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.storage import InMemoryStore
from tqdm.autonotebook import tqdm, trange

### Environment Variables

In [20]:
# AEBS PDFs
path1 = "/kaggle/input/pdffiles/GB AEBS.pdf"
path2 = "/kaggle/input/pdffiles/UN AEBS.pdf"
# Light PDFs
path3 = "/kaggle/input/pdffiles/GB Lighting installation.pdf"
path4 = "/kaggle/input/pdffiles/R048r12e.pdf"

# LLMs
mixtral = ChatGroq(groq_api_key ="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", model = 'mixtral-8x7b-32768', temperature=0.05)
llama3 = ChatGroq(groq_api_key ="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", model = 'llama3-70b-8192', temperature=0.05)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
large_text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300, add_start_index=True)

# For RAG1
template1 = """
Please answer the following question based only on the information provided in the context below.
The answer should be highly-detailed and well-sturctured.
If possible, refer to specific sections number within the context (e.g., "According to section 4.1.2,...").
Do not begin your response with phrases like "Based on the provided context, the answer to the question is:".
If the context does not contain information related to the question, explicitly state that there is no relevant information in the provided context.

CONTEXT: {context}

QUESTION: {question}
"""
template2 = """
Please provide a detailed and well-structured response to the question below, using only the information provided in the context.
Where applicable, refer to specific section numbers within the context (e.g., "According to section 4.1.2,...").
Avoid introductory phrases like "Based on the provided context, the answer to the question is:".
If there is no relevant information in the provided context, explicitly state that there is no pertinent information available.

CONTEXT: {context}

QUESTION: {question}
"""

prompt = PromptTemplate(template=template2, input_variables=["question", "context"])

# For CRAG1
comparison_template = """
We have provided a question and their two answers.
Generate a comparison section without a heading which includes whether both answer are same or partially same or different. If they are paritially same, then what is same and what is different. This comparison is based on the answers generated from both the contexts.
Accuracy and precision are crucial for this task.

QUESTION: {question}

ANSWER 1: {answer1}

ANSWER 2: {answer2}
"""

# For CRAG2
def get_comparision_prompt(query, context1, context2):
    comparison_template = """
    Response in three sections
    
    ANSWER 1: This is firts section, here answer the question form the context 1.
    
    ANSWER 2: This is second section, here answer the question form the context 2.
    
    COMPARISON: This is third section, here answer whether both answer are same or partially same or different. If they are paritially same, then what is same and what is different.
    This section is completely based on answer generated in first and second section.
    
    Please answer the question solely based on the provided context.
    If you can't answer any of the both questions from their context then just tell that there is no answer in that context.
    This is very important for my life, be very precised and accurate in answering the question and also in comparison..

    QUESTION: {question}

    CONTEXT1: {context1}

    CONTEXT2: {context2}
    """
    comparison_prompt = comparison_template.format(context1 = context1,context2 = context2, question = query)
    return comparison_prompt

In [5]:
# Embedding Model
# embedding_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en",
#                                model_kwargs={'device': 'cuda'},
#                                encode_kwargs={'normalize_embeddings': False})

# Parsing Text

In [12]:
embedding_model= FastEmbedEmbeddings()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
def embed_texts(texts):
    return FastEmbedEmbeddings.embed_documents(embedding_model,texts = texts)

def get_header_footer_lines(pdf_path, threshold=0.71):
    with pdfplumber.open(pdf_path) as pdf:
        random_page_nos = random.sample(range(5, len(pdf.pages)), 10)
        
        avg_similarity = 1
        header_lines = -1
        
        while avg_similarity > threshold and header_lines < 4:
            header_lines += 1
            five_lines = []
            
            for page_no in random_page_nos:
                lines = pdf.pages[page_no].extract_text().split('\n')
                if len(lines) > header_lines:
                    five_lines.append(lines[header_lines])
            similarities = cosine_similarity(embed_texts(five_lines))
            avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])
            
        avg_similarity = 1
        footer_lines = -1
        
        while avg_similarity > threshold and footer_lines < 4:
            footer_lines += 1
            five_lines = []
            
            for page_no in random_page_nos:
                lines = pdf.pages[page_no].extract_text().split('\n')
                if len(lines) > footer_lines:
                    five_lines.append(lines[-(footer_lines+1)])
            similarities = cosine_similarity(embed_texts(five_lines))
            avg_similarity = np.mean(similarities[np.triu_indices(len(similarities), k=1)])
            
        return header_lines, footer_lines
    
def extract_text(pdf_path):
    header_lines, footer_lines = get_header_footer_lines(pdf_path)
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                lines = page_text.split('\n')
                if lines:
                    page_text_without_header_footer = '\n'.join(lines[header_lines:-footer_lines])
                    text += page_text_without_header_footer + '\n'
        return text
  

# Get vectorstore

In [49]:
# for PyPDFLoader
def get_vectorstore(path):
    docs = PyPDFLoader(path).load_and_split(text_splitter)
    vectorstore = FAISS.from_documents(docs, embedding_model)
    return vectorstore

In [16]:
# for pdfplumber
def get_vectorstore_2(path):
    splitted_texts = text_splitter.split_text(extract_text(path))
    splitted_docs = [Document(text) for text in splitted_texts if text.strip()]
    vectorstore = FAISS.from_documents(splitted_docs, embedding_model)
    return vectorstore

In [26]:
vectorstore1 = get_vectorstore_2(path2)

IndexError: list index out of range

### PDF 1

In [28]:
vectorstore1 = get_vectorstore_2(path1)

In [30]:
retriever1 = vectorstore1.as_retriever()
qa_chain = RetrievalQA.from_llm(llm=llama3, retriever=retriever1, prompt= prompt)
display(Markdown(qa_chain.invoke("Explain the test procedures in detail, in details.")["result"]))

Based on the provided context, the test procedures can be explained as follows:

**Test Procedure 1: Normal Operation Test**

According to section A.2.2.1, the test procedure involves testing the AEBS under normal operation conditions to determine its normal operation level. This test is conducted under non-fault state of the vehicle system, as per the function concept described in section A.2.2.1.

**Test Procedure 2: Fault State Test**

According to section A.2.4, the test procedure involves simulating the influence of internal faults of assemblies through the application of relevant output signals to electronic/electric assemblies or mechanical assemblies of AEBS. This test is conducted to inspect the reaction of AEBS in case of failure of a single assembly.

The test procedure involves the following steps:

* Apply relevant output signals to electronic/electric assemblies or mechanical assemblies of AEBS to simulate internal faults.
* Observe and record the reaction of AEBS in case of failure of a single assembly.
* Compare the results with the conclusion of the function safety concept described in section A.2.4.
* Describe the sufficiency of relevant safety concepts and their implementation effects.

Note that there is no additional information available in the provided context regarding the specific test methods, tools, or equipment required for these test procedures.

In [55]:
retriever1.invoke("Explain the test procedures in detail, in details.")

[Document(page_content='functions of “system” and realization modes, including control/implementation\ntheory.\nProvide list of inputted and collected variants of system and indicate the\nspecified operating range. And the safety philosophy to return to basic\ndipped-beam function (level C) operation.\nManufacturer shall explain the expression of system function and safety\nphilosophy. Document shall be brief, and shall demonstrate its benefit for\nrelevant design and development by using terminology in the field.\nFor the sake of demand of cyclic technical verification, it is necessary to provide\ndocument to describe how to inspect current operat ing status of “system”.\nThis document is used as reference document for conformity inspection.\n5.22.9.2.4 In order to inspect whether adaptive main-beam function will cause discomfort,\ndistraction of attention or glare to driver, vehicles passing each other or forward\nvehicle, it is necessary to perform road test as per the requirements 

In [32]:
retriever2 = vectorstore1.as_retriever(search_kwargs={"k": 6})
qa_chain = RetrievalQA.from_llm(llm=llama3, retriever=retriever2, prompt= prompt)
display(Markdown(qa_chain.invoke("Explain the test procedures in detail, in details.")["result"]))

The test procedures are outlined in section A.3 Confirmatory and test. There are two main test procedures: confirming the function concept of AEBS and confirming the function safety concept of AEBS.

**Confirming the Function Concept of AEBS (A.3.a)**

The test procedure involves performing a function test under non-fault state of the vehicle system as per the function concept described in A.2.2.1. This test aims to determine the normal operation level of AEBS. The test procedure does not specify any special test procedure, and it is assumed that the test will be conducted according to the standard test procedures.

**Confirming the Function Safety Concept of AEBS (A.3.b)**

The test procedure involves simulating the influence of internal faults of assemblies through the application of relevant output signals to electronic/electric assemblies or mechanical assemblies of AEBS. This test aims to inspect the reaction of AEBS in case of failure of a single assembly.

The test procedure requires:

* Simulating internal faults of assemblies
* Applying relevant output signals to electronic/electric assemblies or mechanical assemblies of AEBS
* Inspecting the reaction of AEBS in case of failure of a single assembly
* Confirming that the results are identical with the conclusion of the function safety concept
* Describing the sufficiency of relevant safety concepts and their implementation effects

There is no additional information available in the provided context regarding the test procedures.

In [34]:
# for pdfplumber, larger chunks
def get_vectorstore_3(path):
    splitted_texts = large_text_splitter.split_text(extract_text(path))
    splitted_docs = [Document(text) for text in splitted_texts if text.strip()]
    vectorstore = FAISS.from_documents(splitted_docs, embedding_model)
    return vectorstore

vectorstore3 = get_vectorstore_3(path1)

In [35]:
retriever3 = vectorstore3.as_retriever()
qa_chain = RetrievalQA.from_llm(llm=llama3, retriever=retriever3, prompt= prompt)
display(Markdown(qa_chain.invoke("Explain the test procedures in detail, in details.")["result"]))

The test procedures for the Advanced Emergency Braking System (AEBS) of commercial vehicles are outlined in Section 5 of the context. Here is a detailed explanation of the test procedures:

**Test Conditions (Section 5.1)**

* The test environment shall comply with the stipulations of 5.1.2 in GB 12676-2014.
* The horizontal visibility range shall allow the target to be observed throughout the test.

**Vehicle Conditions (Section 5.2)**

* The vehicle shall be tested in a condition of load specified by the manufacturer.
* No specific requirements are mentioned for vehicle speed, acceleration, or other parameters.

**Test Methods (Section 5.6 to 5.8)**

* **Test for Warning Signal after System Failure (Section 4.4 and 5.6)**
	+ The test shall be performed to verify that the constant ON optical warning signal is activated not later than 10s after the vehicle has been driven at a speed more than 15km/h.
	+ The vehicle shall maintain its failure warning status after ignition switch 'off' ignition 'on' cycle with the vehicle stationary as long as the failure exists.
* **Test for Interruption by the Driver (Section 4.5 and 5.7)**
	+ The test shall be performed to verify that the driver can interrupt the early warning phase and emergency braking phase.
	+ The test shall include scenarios where the driver initiates a positive action (e.g., depression of accelerator pedal, activation of direction indicator, etc.) to interrupt the AEBS.
* **Test for Prevention of False Reaction (Section 4.6 and 5.8)**
	+ The test shall be performed to verify that the AEBS does not send collision early warning and does not activate emergency braking function.

**Additional Requirements**

* **Validation Test (Section A.2.2)**
	+ The test shall be performed to verify the normal operation and failure mode of AEBS.
	+ The test shall include inspection of the work state under normal operation and failure mode of AEBS.
* **Function Safety Concept (Section A.2.4)**
	+ The test shall be performed to verify the function safety concept of AEBS.
	+ The test shall include simulation of internal fault of assembly through application of relevant output signal to electronic/electric assembly or mechanical assembly of AEBS.

Note that the context does not provide detailed information on the specific test procedures, such as the test scenarios, test speeds, or test distances. It is assumed that these details are provided in the referenced standards, such as GB 12676-2014 and GB/T 34590-2017.

In [36]:
retriever3.invoke("Explain the test procedures in detail, in details.")

[Document(page_content='including random hardware failure and systematic failure, and has applied\nengineering practice of relevant fields. See GB/T 34590.5-2017.\nc) In order to support validation test, describe how to inspect the work state under\nnormal operation and failure mode of AEBS.\nA.2.2 Definitions for relevant items\nA.2.2.1 It is necessary to describe function concept of relevant items and provide description\nlist of functions. Relevant item of AEBS may include environmental perception\nsystem, control system, execution system, driver information interaction system, etc.\nNote: Describe perceivable functions at complete vehicle level and refine.\nA.2.2.2 It is necessary to define scope of relevant items, clarify systems and elements\nbelonging to relevant items, and identify external systems or elements which have\ninteractive relation with it.\nA.2.2.3 It is necessary to define the operation condition and restrictions/limits of relevant\nitems, and describe boundary of 

## RAG 1
- Using FAISS retriever
- Adv. any k
- RetrievalQA chain

In [ ]:
%%time
retriever1 = get_vectorstore(path4).as_retriever(search_kwargs={"k": 6})

In [ ]:
def RAG1(query):
    qa_chain = RetrievalQA.from_llm(llm=llama3, retriever=retriever1, prompt= prompt)
    return qa_chain.invoke(query)['result']

In [47]:
# Lightning
queries4 = ["Whats the difference between Grouped and Combined lamps?", "Can dipped-beam headlamp and main-beam headlamp for front lighting system?", "what is color of End Outline marker lamp?", "Can yellow lamp used as front fog lamp?", "Can red color light placed in the front of the vehicle?", "Can white light can be placed at the back of the vehicle?", "What are 1,1,a,1b,2a,2b,5,6 in direction indicator lamps?", "is cornering lamp mandatory?", "does reflective tape come under light and light signalling?", "standard weight of a person for testing?","can dipped beam uses as a main beam?", "what are the light functions to be kept rear of the vehicle?", "What lamp should be fitted for passenger vehicles?"]

In [ ]:
data = []
for query in queries4:
    print("#",query)
    response = RAG1(query) 
    display(Markdown(response))
    print("---------------------------------------------------------------------")
    data.append({"query": query, "response": response})

In [ ]:
df = pd.DataFrame(data)
df.to_excel('query_responses10.xlsx', index=False)

# RAG2
- using PDR
- Fixed k = 4

In [ ]:
def PDR(path):
    documents = PyPDFLoader(path).load()
    combined_text = "\n".join(document.page_content for document in documents)
    document = [Document(page_content=combined_text, metadata={"source": path})]

    retriever = ParentDocumentRetriever(vectorstore=Chroma(collection_name="full_documents", embedding_function=FastEmbedEmbeddings()),
                                        docstore=InMemoryStore(),
                                        child_splitter=RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100),
                                        parent_splitter=RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100))
    
    retriever.add_documents(document, ids=None)
    return retriever

In [ ]:
%%time
retriever2 = PDR(path4)

In [ ]:
def RAG2(query):
    qa_chain = RetrievalQA.from_llm(llm=llama3, retriever=retriever2, prompt= prompt)
    return qa_chain.invoke(query)['result']

In [ ]:
data = []
for query in queries4:
    print("#",query)
    response = RAG2(query) 
    print(response)
    print("---------------------------------------------------------------------")
    data.append({"query": query, "response": response})

In [ ]:
# Save responses and export
df = pd.DataFrame(data)
df.to_excel('query_responses10.xlsx', index=False)

# CRAG 1
- RAG for comparison
- by FAISS Retrieval
- by calling LLM 3 times
- chain - RetrievalQA

In [ ]:
%%time
retriever1 = get_vectorstore(path1).as_retriever(search_kwargs={"k": 6})
retriever2 = get_vectorstore(path2).as_retriever(search_kwargs={"k": 6})

In [ ]:
def CRAG1(query):
    llm = llama3
    
    qa_chain1 = RetrievalQA.from_llm(llm=llm, retriever=retriever1, prompt= prompt,)
    qa_chain2 = RetrievalQA.from_llm(llm=llm, retriever=retriever2, prompt= prompt)

    answer1 = qa_chain1.invoke(query)['result']
    answer2 = qa_chain2.invoke(query)['result']
    
    comparison_prompt = comparison_template.format(question = query,answer1 = answer1, answer2 = answer2)
    comparison = llm.invoke(comparison_prompt).content
    
    response = f"**ANSWER 1**: {answer1}\n\n**ANSWER 2**: {answer2}\n\n**COMPARISION**: {comparison}"
    return response

In [ ]:
# AEBS
queries2 = ["Explain the test procedures in detail, in details.", "What are warning indications, in details."]
for query in queries2:
    display(Markdown(CRAG1(query)))

# CRAG 2
- by PDR
- 3 LLM calls

In [ ]:
retriever1 = PDR(path1)
retriever2 = PDR(path2)

In [ ]:
def CRAG2(query):
    context1 = get_context(query, retriever1)
    context2 = get_context(query, retriever2)

    qa_prompt1 = qa_template.format(question = query,context = context1)
    qa_prompt2 = qa_template.format(question = query,context = context2)
    
    answer1 = llama3.invoke(qa_prompt1).content
    answer2 = llama3.invoke(qa_prompt2).content
    
    comparison_prompt = comparison_template.format(question = query,answer1 = answer1, answer2 = answer2)
    comparison = llm.invoke(comparison_prompt).content
    
    response = f"**ANSWER 1**: {answer1}\n\n**ANSWER 2**: {answer2}\n\n**COMPARISION**: {comparison}"
    return response

# CRAG 3
- Vectostore to context from scratch`
- Dis. - 1 prompt, 1 LLM call, too much load on geneartion

In [ ]:
def get_context(query, vectorstore):
    retrieved_docs = vectorstore.similarity_search_with_relevance_scores(query, k = 6)
    context = ""
    for doc in retrieved_docs:
        context += doc.page_content
    return context

In [ ]:
vectorstore1 = get_vectorstore(path1)
vectorstore2 = get_vectorstore(path2)

In [ ]:
def CRAG3(query):
    context1 = get_context(query, retriever1)
    context2 = get_context(query, retriever2)
    
    prompt = get_comparision_prompt(query, context1, context2)
    
    return llama3.invoke(prompt).content

In [ ]:
#AEBS
queries1 = ["Explain the test procedures in detail.","When collision early warning signal shall be sent?","What are warning indications, in details.","What AEBS should do in vehicle ignition?","The total speed reduction of the subject vehicle at the time of the collision with the stationary target shall be not less than how many kilometers per hour?"]

In [ ]:
for query in queries1:
    print("#",query)
    display(Markdown(CRAG3(query)))

### Some notes

In [ ]:
## FAISS
# both dont have k as a parameters
# Both returns list of documents retrievd
            
# retriever1.get_relevant_documents(query) #deprceated
# retriever1.invoke(query) #use this instead

In [ ]:
retriever.invoke

In [ ]:
retriever.get_relevant_documents("degrees on a sphere", k =1)

In [ ]:
import inspect
signature = inspect.signature(retriever.invoke)

for param_name, param in signature.parameters.items():
    print(f"Parameter: {param_name}")
    print(f"  Default: {param.default}")
    print(f"  Annotation: {param.annotation}")
    print()

In [ ]:
import inspect
signature = inspect.signature(retriever.get_relevant_documents)

for param_name, param in signature.parameters.items():
    print(f"Parameter: {param_name}")
    print(f"  Default: {param.default}")
    print(f"  Annotation: {param.annotation}")
    print()

In [ ]:
import inspect
signature = inspect.signature(retriever1.get_relevant_documents)

for param_name, param in signature.parameters.items():
    print(f"Parameter: {param_name}")
    print(f"  Default: {param.default}")
    print(f"  Annotation: {param.annotation}")
    print()

In [ ]:
import inspect
signature = inspect.signature(get_vectorstore(path4).as_retriever)

for param_name, param in signature.parameters.items():
    print(f"Parameter: {param_name}")
    print(f"  Default: {param.default}")
    print(f"  Annotation: {param.annotation}")
    print()
    

In [ ]:
#multivector embeddings have best performance for retrieval https://www.rungalileo.io/blog/mastering-rag-how-to-select-an-embedding-model

In [ ]:
print(llama3.invoke("Do you know about how Retrieval augmented generation works? You(llm) will be provided with some context from the whole context(pdf) to generate the answer, but the problem is sometimses the context extracted for you is not very relevant but we can't be sure that whether it is problem of retriever or not(It may possible that relevant information is present in the pdf , but retrievr was not able to extract them). So if the context is irravalent , you can't say the pdf doesn't provide infomation about the query. did you understand what I am saying. If I tell you in the prompt that the llm(you) are used in a rag implementation, will you able to act accordingly? Will it improve the responses?. Write a good prompt for the same issue (for the same issue i mentioned before), insuer that prompt is short and understandable by the llm").content)

In [ ]:
print(llama3.invoke("Telling a llm that he(llm) is being used in a Retrieval augmented generation implementation. How thsi will affect the performanec of behaviour of llm").content)

In [ ]:
# Optional way for RAG2# def get_context(query, retriever):
#     retrieved_docs = retriever.get_relevant_documents(query)
#     context = " ".join([doc.page_content for doc in retrieved_docs])
#     return context

# def RAG2(query):
#     context = get_context(query,retriever1)
#     qa_prompt = template2.format(question = query,context = context)
#     return llama3.invoke(qa_prompt).content

- llama3 could be taken from groq or ollama
- Possible raesons for latency in pdr-llama2-chromadb code
  - llama2 downloaded in local computer
  - slow parent document retriever
  - small cpu
- Giskard for evaluation
  - By default uses gpt4 but llama3 also could be connected
  - it finds contexts from pdf/url and generates question, further context is matched to the RAG response for evaluating RAG.
- Use an image processing model (e.g., CLIP, a Vision Transformer, or a CNN) to convert images into embeddings or textual descriptions.